### Example 1: Fitting the ARMAX model and estimating timescales

**1. Load and Preprocess Data**. First, we load our data from a .mat file and convert it to a `NeuronData` object, the data format used for modeling. 

In [16]:
from scipy.io import loadmat
from neurotime.NeuronData import NeuronData
from neurotime.data_utils import SignalTimes, SignalValues

data_path = './example_data/example_neuron_data.mat'
mat_contents = loadmat(data_path)
neuron_data_mat = mat_contents['NeuronData'][0]

tst = neuron_data_mat['trial_start_times'][0]
tet = neuron_data_mat['trial_end_times'][0]
spike_times = neuron_data_mat['spike_times'][0]

st = neuron_data_mat['signal_times'][0]
sv = neuron_data_mat['signal_values'][0]

signal_times = SignalTimes()
signal_values = SignalValues()

signal_times.reward = st['reward'][0][0]
signal_values.reward = sv['reward'][0][0]
signal_times.choice_side = st['choice_side'][0][0]
signal_values.choice_side = sv['choice_side'][0][0]
signal_times.choice_stimulus = st['choice_stimulus'][0][0]
signal_values.choice_stimulus = sv['choice_stimulus'][0][0]

# construct the neuron data object
neuron_data = NeuronData(trial_start_times = tst, trial_end_times = tet, 
                            spike_times=spike_times, signal_times=signal_times, 
                            signal_values=signal_values)


**2. Define ARMAX Model.** Next, we construct a `ModelDefinition` object that describes the composition of the model. Then we use the `ModelDefinition` object to construct an `ARMAXModel`.

In [17]:
from neurotime.ModelDefinition import ModelDefinition
from neurotime.ARMAXModel import ARMAXModel

# here, we add 3 task relevant, 2 autoregressive, and 2 exogenous terms to the model
model_definition = ModelDefinition()
model_definition.addTRComponent(name = 'reward')
# model_definition.addTRComponent(name = 'choice_side')
# model_definition.addTRComponent(name = 'choice_stimulus')
# model_definition.addARComponent(name = 'intrinsic')
# model_definition.addARComponent(name = 'seasonal')
# model_definition.addExoComponent(name = 'reward')
# model_definition.addExoComponent(name = 'choice_side')
# model_definition.addExoComponent(name = 'choice_stimulus')

# now, we construct the ARMAX model
armax_model = ARMAXModel(model_definition=model_definition)

**3. Fit ARMAX Model.** Next, we convert `NeuronData` to $X$ and $y$ variables to input into the model. Then, we fit the ARMAX model using thoose variables. 

In [18]:
# first, generate input for the model
# X, y are inputs for the fit function, and i_size, and s_size are the size of intrinsic 
#   and seasonal bin for later timescale estimation
X, y, i_size, s_size = neuron_data.to_armax_input(model = armax_model, model_definition = model_definition, bin_size = 50)

# next, call fit function
armax_model.fit(X, y)

# finally, we report fitting results
r_sq = armax_model.score(X, y)
print(f'Model Performance: R^2={r_sq}')

params = armax_model.get_params(intrinsic_size=i_size, seasonal_size=s_size)
print(f'Model parameters: {params}')

removed 0/22180 rows because they contained a NaN predictor.
Model Performance: R^2=0.00015475794624623948
Model parameters: {'tr_reward': -0.008390107544308882}
